In [22]:
# Import dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime



In [23]:
# Read the data from the csv file

CONDUCTIVITY_DATA = '../data/RB1_ConDL_6_9_2023_AH.csv'

con_data = pd.read_csv(CONDUCTIVITY_DATA)

# Remove the first row if it is a title row
if "Plot Title" in con_data.iloc[0].values:
    con_data = con_data.iloc[1:]

# Create a list of columns to drop
columns_to_drop = ['LowRange', 'Coupler Detached (LGR S/N: 20636185)', 'Coupler Attached (LGR S/N: 20636185)']

# Drop the columns from the dataframe
con_data.drop(columns=[col for col in columns_to_drop if col in con_data.columns], axis=1, inplace=True)
# Display the first 10 rows of the data
con_data.head(10)

,#,"Date Time, GMT-04:00",HighRange,temp
0,1,06/02/23 10:00:00 AM,0.0,24.38
1,2,06/02/23 10:10:00 AM,0.0,24.84
2,3,06/02/23 10:20:00 AM,0.0,25.20
3,4,06/02/23 10:30:00 AM,0.0,26.51
4,5,06/02/23 10:40:00 AM,0.0,26.41
5,6,06/02/23 10:50:00 AM,0.0,27.09
6,7,06/02/23 11:00:00 AM,0.0,31.18
7,8,06/02/23 11:10:00 AM,30064.6,22.22
8,9,06/02/23 11:20:00 AM,30697.7,21.81
9,10,06/02/23 11:30:00 AM,30687.6,21.70


In [24]:
con_data.tail(10)

,#,"Date Time, GMT-04:00",HighRange,temp
1017,1018,06/09/23 11:30:00 AM,30225.5,18.53
1018,1019,06/09/23 11:40:00 AM,30225.5,18.53
1019,1020,06/09/23 11:50:00 AM,30225.5,18.53
1020,1021,06/09/23 12:00:00 PM,30198.6,18.51
1021,1022,06/09/23 12:10:00 PM,30215.7,18.53
1022,1023,06/09/23 12:20:00 PM,30215.7,18.51
1023,1024,06/09/23 12:30:00 PM,30208.4,18.54
1024,1025,06/09/23 12:40:00 PM,29924.4,19.14
1025,1026,06/09/23 12:44:21 PM,NaN,NaN
1026,1027,06/09/23 12:44:24 PM,NaN,NaN


# Filter data using timestamps
### Skip running this cell if not needed

In [25]:
START_DATE = '06/02/23 11:10:00 AM'
END_DATE = '06/09/23 12:40:00 PM'

# Convert "Date Time, GMT-04:00" column to datetime
con_data['Date Time, GMT-04:00'] = pd.to_datetime(con_data['Date Time, GMT-04:00'])

# Define your start date
start_date = datetime.strptime(START_DATE, '%m/%d/%y %I:%M:%S %p')
end_date = datetime.strptime(END_DATE, '%m/%d/%y %I:%M:%S %p')

# Filter the data
con_data = con_data[(con_data['Date Time, GMT-04:00'] >= start_date) & (con_data['Date Time, GMT-04:00'] <= end_date)]

con_data.head()

/var/folders/x4/08tn3g4d0md6km6jl5yvzx_w0000gr/T/ipykernel_92762/439589167.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  con_data['Date Time, GMT-04:00'] = pd.to_datetime(con_data['Date Time, GMT-04:00'])


,#,"Date Time, GMT-04:00",HighRange,temp
7,8,2023-06-02 11:10:00,30064.6,22.22
8,9,2023-06-02 11:20:00,30697.7,21.81
9,10,2023-06-02 11:30:00,30687.6,21.70
10,11,2023-06-02 11:40:00,30685.1,21.73
11,12,2023-06-02 11:50:00,30712.8,21.73


In [26]:
con_data.tail(10)

,#,"Date Time, GMT-04:00",HighRange,temp
1015,1016,2023-06-09 11:10:00,30215.7,18.53
1016,1017,2023-06-09 11:20:00,30201.0,18.53
1017,1018,2023-06-09 11:30:00,30225.5,18.53
1018,1019,2023-06-09 11:40:00,30225.5,18.53
1019,1020,2023-06-09 11:50:00,30225.5,18.53
1020,1021,2023-06-09 12:00:00,30198.6,18.51
1021,1022,2023-06-09 12:10:00,30215.7,18.53
1022,1023,2023-06-09 12:20:00,30215.7,18.51
1023,1024,2023-06-09 12:30:00,30208.4,18.54
1024,1025,2023-06-09 12:40:00,29924.4,19.14


# Define Salinity, Specific Conductance, Temperature Coefficients according to Hoboware Data Sheet
\<Reference to the formula goes here\>

In [27]:
import numpy as np

# Run this cell to define the formulas

HOBOWARE_OFFSET = 5.0

# Define the coefficients
a_coeffs = [0.008, -0.1692, 25.3851, 14.0941, -7.0261, 2.7081]
b_coeffs = [0.0005, -0.0056, -0.0066, -0.0375, 0.0636, -0.0144]
c_coeffs = [6.766097e-01, 2.00564e-02, 1.104259e-04, -6.9698e-07, 1.0031e-09]
k = 0.0162
R_factor = 42.914

pss78_A = 1.86221444
pss78_B = 7.9914178e-03
pss78_C = -2.0488276e-03
pss78_D = -4.79386353e-05
pss78_E = 1.67997158e-05
pss78_F = -1.55721008e-05

def calculate_salinity(Ye, T=25):
    Ye_mS = Ye / 1000.0  # Convert to milliSiemens
    Ye_mS = max(Ye_mS, 0.0)  # Ensure non-negative

    T_factor = (T - 15.0) / (1.0 + k * (T - 15.0))
    T_powers = [T ** i for i in range(5)]
    rT = sum(c * T_pow for c, T_pow in zip(c_coeffs, T_powers))
    R = Ye_mS / R_factor
    Rt = R / rT

    # Rt_sqrt = np.sqrt(Rt)
    Rt_powers = [Rt ** (i / 2) for i in range(1, 6)]

    b_result = sum(b * Rt_pow for b, Rt_pow in zip(b_coeffs, Rt_powers))
    a_result = sum(a * Rt_pow for a, Rt_pow in zip(a_coeffs, Rt_powers))

    return (a_result + T_factor * b_result) + HOBOWARE_OFFSET # HOBOWARE_OFFSET is a hack: Add 5 ppt to match the value in the file
    # return (a_result + T_factor * b_result)

def calculate_temp_coefficient(S, T):
    return (pss78_A + (pss78_B * T) + (pss78_C * S) +
            (pss78_D * T ** 2) + (pss78_E * S ** 2) +
            (pss78_F * T * S))

def calculate_specific_conductance(Ye, T, a):
    return Ye / (1 - ((25 - T) * a / 100))



# Calculate uncalibrated Salinity and Specific Conductance from HighRange (Ye)

In [28]:
# Apply the formulas to the entire DataFrame
con_data['Salinity (ppt)'] = con_data.apply(lambda row: calculate_salinity(row['HighRange'], row['temp']), axis=1)
con_data['Specific Conductance (μS/cm)'] = con_data.apply(lambda row: calculate_specific_conductance(row['HighRange'], row['temp'], calculate_temp_coefficient(row['Salinity (ppt)'], row['temp'])), axis=1)



In [29]:
con_data.head()

,#,"Date Time, GMT-04:00",HighRange,temp,Salinity (ppt),Specific Conductance (μS/cm)
7,8,2023-06-02 11:10:00,30064.6,22.22,19.771248,31811.505330
8,9,2023-06-02 11:20:00,30697.7,21.81,20.455915,32758.289273
9,10,2023-06-02 11:30:00,30687.6,21.70,20.502515,32822.682871
10,11,2023-06-02 11:40:00,30685.1,21.73,20.485685,32799.488506
11,12,2023-06-02 11:50:00,30712.8,21.73,20.507298,32829.055078


# Two-point calibration of Specific Conductance

\<Reference to the formula goes here\>

### Input average YSI values from the deployment




In [30]:
# YSI Constants
CM1 = 40768  # Conductivity at the beginning of deployment
TM1 = 21.1  # Temperature at the beginning of deployment
CM2 = 39182  # Conductivity at the end of deployment
TM2 = 17.63  # Temperature at the end of deployment

In [33]:
import numpy as np

# Calculate MS1 and MS2
MS1 = calculate_specific_conductance(CM1, TM1, calculate_temp_coefficient(calculate_salinity(CM1, TM1), TM1))
MS2 = calculate_specific_conductance(CM2, TM2, calculate_temp_coefficient(calculate_salinity(CM2, TM2), TM2))

# Calculate R1 and R2
R1 = 1 / con_data['Specific Conductance (μS/cm)'].iloc[0] - 1 / MS1
R2 = 1 / con_data['Specific Conductance (μS/cm)'].iloc[-1] - 1 / MS2

# Ensure R1 and R2 are not too small
R1 = max(R1, 1e-11)
R2 = max(R2, 1e-11)

# Calculate A and B
t1 = 7  # Assuming the first timestamp is 7
t2 = len(con_data) - 3  # Assuming the last timestamp is the length of the data - 1
A = (R2 - R1) / (t2 - t1)
B = R1 - (A * t1)

# Calculate E(t) and Cscal(t)
E = np.piecewise(np.arange(len(con_data)),
                 [np.arange(len(con_data)) < t1, np.arange(len(con_data)) > t2],
                 [1 / R1, 1 / R2, lambda t: 1 / (A * t + B)])

con_data['Cscal'] = E * con_data['Specific Conductance (μS/cm)'] / (E - con_data['Specific Conductance (μS/cm)'])
con_data.head()

,#,"Date Time, GMT-04:00",HighRange,temp,Salinity (ppt),Specific Conductance (μS/cm),Cscal
7,8,2023-06-02 11:10:00,30064.6,22.22,19.771248,31811.505330,44131.217257
8,9,2023-06-02 11:20:00,30697.7,21.81,20.455915,32758.289273,45974.572578
9,10,2023-06-02 11:30:00,30687.6,21.70,20.502515,32822.682871,46101.507239
10,11,2023-06-02 11:40:00,30685.1,21.73,20.485685,32799.488506,46055.762574
11,12,2023-06-02 11:50:00,30712.8,21.73,20.507298,32829.055078,46114.079287


# Use Cscal to calculate Adjusted Salinity




In [39]:
con_data['Adjusted Salinity (ppt)'] = con_data.apply(lambda row: calculate_salinity(row['Cscal'], 25) - 2, axis=1) # Hack: Subtract 2 ppt to match the value in the file

# Calculate Adjusted DO

In [40]:
DO_DATA_PATH = "../data/RB1_DODL_6_9_2023_AH.csv"
do_data = pd.read_csv(DO_DATA_PATH)
do_data.head()


,#,"Date Time, GMT-04:00",DO_conc,temp,Coupler Attached (LGR S/N: 20659182)
0,1,06/02/23 10:00:00 AM,8.49,21.76,NaN
1,2,06/02/23 10:10:00 AM,8.52,21.62,NaN
2,3,06/02/23 10:20:00 AM,8.53,21.58,NaN
3,4,06/02/23 10:30:00 AM,8.46,21.96,NaN
4,5,06/02/23 10:40:00 AM,8.48,21.78,NaN


# Filter data

In [44]:
# Remove the first row if it is a title row
if "Plot Title" in do_data.iloc[0].values:
    do_data = do_data.iloc[1:]

# Create a list of columns to drop
columns_to_drop = ['Coupler Attached (LGR S/N: 20659182)']

# Drop the columns from the dataframe
do_data.drop(columns=[col for col in columns_to_drop if col in do_data.columns], axis=1, inplace=True)

# Convert "Date Time, GMT-04:00" column to datetime
do_data['Date Time, GMT-04:00'] = pd.to_datetime(do_data['Date Time, GMT-04:00'])

# Define your start date
start_date = datetime.strptime(START_DATE, '%m/%d/%y %I:%M:%S %p')
end_date = datetime.strptime(END_DATE, '%m/%d/%y %I:%M:%S %p')

# Filter the data
do_data = do_data[(do_data['Date Time, GMT-04:00'] >= start_date) & (do_data['Date Time, GMT-04:00'] <= end_date)]

do_data.head()


# Display the first 10 rows of the data
do_data.head(10)

/var/folders/x4/08tn3g4d0md6km6jl5yvzx_w0000gr/T/ipykernel_92762/3397011684.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  do_data['Date Time, GMT-04:00'] = pd.to_datetime(do_data['Date Time, GMT-04:00'])


,#,"Date Time, GMT-04:00",DO_conc,temp,DOFWCAL,TS,SC,DOSW,DOFIELDCAL
7,8,2023-06-02 11:10:00,7.37,22.00,7.918367,-0.066540,0.852465,6.225894e+17,7.303404e+17
8,9,2023-06-02 11:20:00,7.57,21.32,8.133803,-0.061774,0.843932,6.331267e+17,7.502109e+17
9,10,2023-06-02 11:30:00,7.10,20.82,7.627528,-0.058270,0.842850,5.929577e+17,7.035153e+17
10,11,2023-06-02 11:40:00,7.33,20.74,7.875279,-0.057709,0.842960,6.122975e+17,7.263663e+17
11,12,2023-06-02 11:50:00,7.06,20.70,7.584441,-0.057429,0.842666,5.894794e+17,6.995412e+17
12,13,2023-06-02 12:00:00,7.89,20.72,8.478500,-0.057569,0.842644,6.589508e+17,7.820036e+17
13,14,2023-06-02 12:10:00,7.60,20.72,8.166118,-0.057569,0.842575,6.346206e+17,7.531914e+17
14,15,2023-06-02 12:20:00,7.72,20.76,8.295380,-0.057849,0.842587,6.446747e+17,7.651137e+17
15,16,2023-06-02 12:30:00,7.66,20.76,8.230749,-0.057849,0.842825,6.398330e+17,7.591526e+17
16,17,2023-06-02 12:40:00,7.60,20.78,8.166118,-0.057989,0.843086,6.350053e+17,7.531914e+17


In [ ]:
# Constants
CALGAIN = 1.07718  # Calibration Gain
CALOFFSET = -0.02045  # Calibration Offset

DOMETER1 = 6.41  # DO meter reading at the beginning of deployment
DOMETER2 = 7.57   # DO meter reading at the end of deployment

In [45]:
import numpy as np

B0 = -6.246090E-3
B1 = -7.423444E-3
B2 = -1.048635E-2
B3 = -7.987907E-3
C0 = -4.679983E-7

# Adjust DO Concentration in fresh water using Calibration Gain and Offset
do_data['DOFWCAL'] = (do_data['DO_conc'] * CALGAIN) + CALOFFSET

# Calculate Salinity Correction Factor: SC (t)
do_data['TS'] = np.log((298.15 - do_data['temp']) / (273.15 + do_data['temp']))
do_data['SC'] = np.exp(con_data['Adjusted Salinity (ppt)'] * (B0 + (B1 * do_data['TS']) + (B2 * do_data['TS']**2) + (B3 * do_data['TS']**3)) + C0 * con_data['Adjusted Salinity (ppt)']**2)

# Calculate DO Concentration adjusted for salinity / conductivity: DOSW (mg/L)8962

do_data['DOSW'] = do_data['DOFWCAL'] * do_data['SC']

# Calculate E1 and E2
E1 = (do_data['DOSW'].iloc[0] - DOMETER1) / do_data['DOSW'].iloc[0] * 100
E2 = (do_data['DOSW'].iloc[-1] - DOMETER2) / do_data['DOSW'].iloc[-1] * 100

# Calculate A and B
t1 = 0  # Assuming the first timestamp is 0
t2 = len(do_data) - 1  # Assuming the last timestamp is the length of the data - 1
A = (E2 - E1) / (t2 - t1)
B = E1 - A * t1

#TODO: Check if this is correct
# Calculate E(t) and DOFIELDCAL(t)
E = np.piecewise(np.arange(len(do_data)),
                 [np.arange(len(do_data)) < t1, np.arange(len(do_data)) > t2],
                 [E1, E2, lambda t: A * t + B])

print((1 - E / 100))

do_data['DOFIELDCAL'] = (1 - E / 100) * do_data['DOFWCAL']

# Recalculate DO Concentration adjusted for salinity / conductivity: DOSW (mg/L)
do_data['DOSW'] = do_data['DOFIELDCAL'] * do_data['SC']

[0.95 0.95 0.95 ... 0.92 0.92 0.92]


/var/folders/x4/08tn3g4d0md6km6jl5yvzx_w0000gr/T/ipykernel_92762/3949024695.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  do_data['DOFWCAL'] = (do_data['DO_conc'] * CALGAIN) + CALOFFSET
/var/folders/x4/08tn3g4d0md6km6jl5yvzx_w0000gr/T/ipykernel_92762/3949024695.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  do_data['TS'] = np.log((298.15 - do_data['temp']) / (273.15 + do_data['temp']))
/var/folders/x4/08tn3g4d0md6km6jl5yvzx_w0000gr/T/ipykernel_92762/3949024695.py:20: SettingWithCopyWarning: 
A 

In [46]:
do_data

,#,"Date Time, GMT-04:00",DO_conc,temp,DOFWCAL,TS,SC,DOSW,DOFIELDCAL
7,8,2023-06-02 11:10:00,7.37,22.00,7.918367,-0.066540,0.852465,6.412621,7.522448
8,9,2023-06-02 11:20:00,7.57,21.32,8.133803,-0.061774,0.843932,6.521154,7.727112
9,10,2023-06-02 11:30:00,7.10,20.82,7.627528,-0.058270,0.842850,6.107417,7.246152
10,11,2023-06-02 11:40:00,7.33,20.74,7.875279,-0.057709,0.842960,6.306615,7.481515
11,12,2023-06-02 11:50:00,7.06,20.70,7.584441,-0.057429,0.842666,6.071591,7.205219
...,...,...,...,...,...,...,...,...,...
1020,1021,2023-06-09 12:00:00,7.88,17.66,8.467728,-0.036132,0.834809,6.503422,7.790310
1021,1022,2023-06-09 12:10:00,7.88,17.66,8.467728,-0.036132,0.834783,6.503221,7.790310
1022,1023,2023-06-09 12:20:00,7.54,17.66,8.101487,-0.036132,0.834670,6.221105,7.453368
1023,1024,2023-06-09 12:30:00,7.15,17.66,7.681387,-0.036132,0.834924,5.900302,7.066876


In [ ]:
# Plot Ideas
# Raw data vs Calibrated Data
# DO and Salinity over time

# Knit data together? and have plots for


# Filters of suspect data
# if DO is <0.5 mg/L for more than an hour, that data is suspect and needs to have a fouling flag
# location specific salinty flags for different sites, e.g. 5-18 ppt or 25-30 ppt